## Support 

In [1]:
import sys
import logging

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(format='[%(asctime)s] %(levelname)s - %(message)s',
                        level=logging.INFO)

In [2]:
def pandas_df_to_markdown_table(df,name):
    '''
    Write the df in name to markdown format
    '''
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    df_formatted.to_csv(name,sep="|", index=False)

In [3]:
logging.info('start')

[2020-02-13 09:56:36,293] INFO - start


## Dataset

In [4]:
!mkdir -p data
!wget -q -O data/estim-pop-dep-sexe-gca-1975-2019.xls data https://raw.githubusercontent.com/KhalidCK/tidydata/master/data/estim-pop-dep-sexe-gca-1975-2019.xls

In [5]:
!ls data

estim-pop-dep-sexe-gca-1975-2019.xls


>Chaque année, l'Insee estime la population des régions et des départements (France métropolitaine et DOM) à la date du 1ᵉʳ janvier. Ces estimations annuelles de population sont déclinées par sexe et par âge (quinquennal, classes d'âge).


[ref](https://www.insee.fr/fr/statistiques/1893198)

In [6]:
import pandas as pd

In [7]:
pop2019 = pd.read_excel('data/estim-pop-dep-sexe-gca-1975-2019.xls'
                        ,sheet_name='2019'
                        ,skiprows=4)

In [8]:
pop2019.head()

,Unnamed: 0,Unnamed: 1,0 à 19 ans,20 à 39 ans,40 à 59 ans,60 à 74 ans,75 ans et plus,Total,0 à 19 ans.1,20 à 39 ans.1,40 à 59 ans.1,60 à 74 ans.1,75 ans et plus.1,Total.1,0 à 19 ans.2,20 à 39 ans.2,40 à 59 ans.2,60 à 74 ans.2,75 ans et plus.2,Total.2
0,01,Ain,167720.0,150949.0,179476.0,102788.0,52755.0,653688.0,86359.0,75242.0,89278.0,49523.0,21526.0,321928.0,81361.0,75707.0,90198.0,53265.0,31229.0,331760.0
1,02,Aisne,131435.0,115046.0,137405.0,96000.0,48130.0,528016.0,67391.0,57430.0,68197.0,45907.0,18308.0,257233.0,64044.0,57616.0,69208.0,50093.0,29822.0,270783.0
2,03,Allier,67628.0,61986.0,87232.0,71910.0,44309.0,333065.0,34894.0,31397.0,42594.0,33582.0,16973.0,159440.0,32734.0,30589.0,44638.0,38328.0,27336.0,173625.0
3,04,Alpes-de-Haute-Provence,33883.0,30028.0,43039.0,34523.0,20507.0,161980.0,17721.0,15097.0,20941.0,16884.0,8510.0,79153.0,16162.0,14931.0,22098.0,17639.0,11997.0,82827.0
4,05,Hautes-Alpes,30518.0,28633.0,37887.0,28356.0,16390.0,141784.0,15987.0,14362.0,18843.0,13704.0,6592.0,69488.0,14531.0,14271.0,19044.0,14652.0,9798.0,72296.0


Des colonnes ne sont pas detecté par pandas.La feuille de donnée essaye de créer une agrégation sur des colonnes visuellement.

On donne des noms pertients à ces colonnes en faisant une inspection de la feuille Excel.

In [9]:
pop2019.columns = ["departements","nom_departement"]+list(pop2019.columns[2:])

C'est souvent une bonne idée de trouver une colonne qui pourra faire office de clé (index)

In [10]:
assert len(pop2019.departements) == len(pop2019.departements.unique())

les departements sont usuellement sur 2 ou 3 (DOM) caractères

In [11]:
all(pop2019.departements.str.len() <=3)

False

Quelles sont les departements avec plus de deux 3 digits

In [12]:
pop2019[pop2019.departements.str.len()>3][['departements']]

,departements
96,France métropolitaine
103,France métropolitaine et DOM
105,Source : Insee - Estimations de population (ré...


In [13]:
dep_nom = pop2019[["departements","nom_departement"]].copy()

In [14]:
pop2019=(pop2019
                .loc[pop2019.departements.str.len()<3]
                .drop(columns="nom_departement")
                .set_index('departements'))

In [15]:
pop2019.tail()

,0 à 19 ans,20 à 39 ans,40 à 59 ans,60 à 74 ans,75 ans et plus,Total,0 à 19 ans.1,20 à 39 ans.1,40 à 59 ans.1,60 à 74 ans.1,75 ans et plus.1,Total.1,0 à 19 ans.2,20 à 39 ans.2,40 à 59 ans.2,60 à 74 ans.2,75 ans et plus.2,Total.2
departements,,,,,,,,,,,,,,,,,,
91,359997.0,341768.0,349855.0,173012.0,90195.0,1314827.0,183666.0,167455.0,173901.0,81570.0,35870.0,642462.0,176331.0,174313.0,175954.0,91442.0,54325.0,672365.0
92,396449.0,456842.0,428253.0,207498.0,117046.0,1606088.0,201925.0,220864.0,204127.0,93799.0,43734.0,764449.0,194524.0,235978.0,224126.0,113699.0,73312.0,841639.0
93,480144.0,472709.0,423665.0,194156.0,83803.0,1654477.0,243902.0,230657.0,212905.0,93290.0,34747.0,815501.0,236242.0,242052.0,210760.0,100866.0,49056.0,838976.0
94,353868.0,388992.0,368508.0,184684.0,99157.0,1395209.0,180460.0,188522.0,179365.0,85189.0,37330.0,670866.0,173408.0,200470.0,189143.0,99495.0,61827.0,724343.0
95,355868.0,328200.0,323470.0,162965.0,73418.0,1243921.0,180630.0,157689.0,157630.0,77658.0,28704.0,602311.0,175238.0,170511.0,165840.0,85307.0,44714.0,641610.0


In [16]:
assert all(pop2019.index.str.len() <=3)

Les colonnes sont assignés à 3 groupes en fonction de la portion de la colonne (tous les 6 elements => nouvelle section).

Dans l'ordre d'apres le fichier initial : 'ensemble','hommes','femmes'

In [17]:
from itertools import zip_longest

#https://docs.python.org/3.7/library/itertools.html
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [18]:
elements = ['ensemble','hommes','femmes']
data = {}
for element,cols in zip(elements,list(grouper(pop2019.columns,6))):
    df = pop2019.loc[:,cols]
    df.columns = [col.split('.')[0] for col in df.columns]
    data[element] = df.drop(columns='Total')

In [19]:
data.keys()

dict_keys(['ensemble', 'hommes', 'femmes'])

In [20]:
data["hommes"].head()

,0 à 19 ans,20 à 39 ans,40 à 59 ans,60 à 74 ans,75 ans et plus
departements,,,,,
01,86359.0,75242.0,89278.0,49523.0,21526.0
02,67391.0,57430.0,68197.0,45907.0,18308.0
03,34894.0,31397.0,42594.0,33582.0,16973.0
04,17721.0,15097.0,20941.0,16884.0,8510.0
05,15987.0,14362.0,18843.0,13704.0,6592.0


In [21]:
data["femmes"].head()

,0 à 19 ans,20 à 39 ans,40 à 59 ans,60 à 74 ans,75 ans et plus
departements,,,,,
01,81361.0,75707.0,90198.0,53265.0,31229.0
02,64044.0,57616.0,69208.0,50093.0,29822.0
03,32734.0,30589.0,44638.0,38328.0,27336.0
04,16162.0,14931.0,22098.0,17639.0,11997.0
05,14531.0,14271.0,19044.0,14652.0,9798.0


In [22]:
pandas_df_to_markdown_table(data["ensemble"].head().reset_index(),'peek-csv-table.md')

Le nom des colonnes sont des variables

In [23]:
def to_tidy(df)->pd.DataFrame:
    return df.reset_index().melt(id_vars='departements',var_name="age",value_name="nb")

In [24]:
tidys = {segment:to_tidy(df) for segment,df in data.items()}

In [25]:
tidys['femmes'].head()

,departements,age,nb
0,01,0 à 19 ans,81361.0
1,02,0 à 19 ans,64044.0
2,03,0 à 19 ans,32734.0
3,04,0 à 19 ans,16162.0
4,05,0 à 19 ans,14531.0


In [26]:
france = tidys['ensemble']
france.nb=france.nb.astype(int)

In [27]:
tidy_sample=france.sample(10)

In [28]:
tidy_sample

,departements,age,nb
168,72,20 à 39 ans,117603
321,33,60 à 74 ans,263186
238,46,40 à 59 ans,45140
417,33,75 ans et plus,140304
17,18,0 à 19 ans,63823
286,94,40 à 59 ans,368508
266,74,40 à 59 ans,227906
423,39,75 ans et plus,29251
369,81,60 à 74 ans,75795
397,14,75 ans et plus,69010


In [29]:
pandas_df_to_markdown_table(tidy_sample,'sample-tidy-france.md')

In [30]:
france = france[france.age!="Total"]

In [31]:
france_total_dep = (france[['departements','nb']]
                    .groupby('departements')
                    .sum()
                    .rename(columns={'nb':'total'}))

In [32]:
france_total_dep.head()

,total
departements,
01,653688
02,528016
03,333065
04,161980
05,141784


In [33]:
#vérification, ordre de grandeur ok
france_total_dep.sum()/10**6

[2020-02-13 09:56:39,214] INFO - NumExpr defaulting to 4 threads.


total    64.812052
dtype: float64

In [34]:
pop = "ensemble"

In [35]:
deps = ['59','75','67']

In [36]:
sub = tidys[pop]
sub = sub[sub.departements.isin(deps)]

In [37]:
france[france.departements.isin(['59','75','67'])]

,departements,age,nb
59,59,0 à 19 ans,678949
67,67,0 à 19 ans,258561
75,75,0 à 19 ans,409783
155,59,20 à 39 ans,668218
163,67,20 à 39 ans,283877
171,75,20 à 39 ans,712617
251,59,40 à 59 ans,652073
259,67,40 à 59 ans,303680
267,75,40 à 59 ans,538693
347,59,60 à 74 ans,399542


In [38]:
deps = ['59','75','67']

In [39]:
sub=sub.join(france_total_dep,on="departements")

In [40]:
sub['pourcentage'] = ((sub['nb'] / sub['total'])*100).round(2)

In [41]:
sub.sort_values('departements')

,departements,age,nb,total,pourcentage
59,59,0 à 19 ans,678949,2592185,26.19
155,59,20 à 39 ans,668218,2592185,25.78
251,59,40 à 59 ans,652073,2592185,25.16
347,59,60 à 74 ans,399542,2592185,15.41
443,59,75 ans et plus,193403,2592185,7.46
67,67,0 à 19 ans,258561,1126505,22.95
163,67,20 à 39 ans,283877,1126505,25.20
259,67,40 à 59 ans,303680,1126505,26.96
355,67,60 à 74 ans,182406,1126505,16.19
451,67,75 ans et plus,97981,1126505,8.70


In [42]:
import altair as alt

alt.Chart(sub).mark_bar().encode(
    x="age:O",
    y="pourcentage:Q",
    color="age:N",
    column='departements:N')

alt.Chart(...)